In [91]:
import random
import networkx as nx
import itertools

In [ ]:
#TEST

In [92]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [93]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [94]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [95]:
def generate_population(size,G):
    return [[random.randint(0, 1) for _ in range(len(G.nodes))] for _ in range(size)]

In [96]:
# def generate_population(size, G, centrality_measure='betweenness', coverage=0.3):
#     population = []

#     # Broj centrality-based jedinki (npr. 30%)
#     central_count = int(size * 0.3)

#     # Izračunaj centralnost
#     if centrality_measure == 'betweenness':
#         centrality = nx.betweenness_centrality(G)
#     elif centrality_measure == 'degree':
#         centrality = dict(G.degree())
#     else:
#         raise ValueError("Nepoznata centralnost")

#     sorted_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
#     num_selected = max(1, int(coverage * len(G.nodes)))
#     selected_nodes = [node for node, _ in sorted_nodes[:num_selected]]

#     # Generiši centrality-based jedinke
#     for _ in range(central_count):
#         individual = [1 if i in selected_nodes else 0 for i in range(len(G.nodes))]
#         population.append(individual)

#     # Generiši ostatak populacije nasumično
#     for _ in range(size - central_count):
#         individual = [random.randint(0, 1) for _ in range(len(G.nodes))]
#         population.append(individual)

#     return population

In [97]:
def fitness(individual, G, shortest_paths):
    resolving_set = {i for i, bit in enumerate(individual) if bit == 1}

    if not resolving_set:
        return float("inf")

    unresolved_pairs = 0
    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova

In [98]:
def tournament_selection(population, G, k=3):
    selected = random.sample(population, k)
    return min(selected, key=lambda ind: fitness(ind, G))

In [99]:
def weighted_selection(population, G,shortest_paths):
    fitness_values = [1 / (1 + fitness(ind, G,shortest_paths)) for ind in population]  # Inverzija jer tražimo minimu
    return random.choices(population, weights=fitness_values, k=2)  # Bira 2 roditelja

In [100]:
def crossover(parent1, parent2,G ):
    point = random.randint(1, len(G.nodes) - 1)
    return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

In [101]:
def mutate(individual, mutation_rate=0.05):
    new_individual = individual[:]
    for i in range(len(new_individual)):
        if new_individual[i] == 1 and random.random() < mutation_rate:
            new_individual[i] = 0  # Samo uklanjamo
    return new_individual


In [102]:
# def mutate(individual, mutation_rate):
#     return [bit if random.random() > mutation_rate else 1 - bit for bit in individual]

In [103]:
def genetic_algorithm(G,shortest_paths,pop_size=50, generations=15, crossover_rate=0.7, mutation_rate=0.2):
    population = generate_population(pop_size,G)
    
    for gen in range(generations):
        new_population = []
      
        for _ in range(pop_size // 2):
            # parent1 = tournament_selection(population,G)
            # parent2 = tournament_selection(population,G)
            parent1, parent2 = weighted_selection(population, G,shortest_paths)            
            # Ukrštanje
            if random.random() < crossover_rate:
                child1, child2 = crossover(parent1, parent2,G)
            else:
                child1, child2 = parent1, parent2
            
            # Mutacija
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.extend([child1, child2])

        population = new_population
        
        # Pronalazak najbolje jedinke
        best_solution = min(population, key=lambda ind: fitness(ind, G,shortest_paths))
        best_fitness = fitness(best_solution,G,shortest_paths)

        print(f"Generacija {gen+1}, Najbolji fitnes: {best_fitness}")

       

    # Prikaz optimalnog rešenja
    optimal_set = {i for i, bit in enumerate(best_solution) if bit == 1}
    print("\nOptimalni rešavajući skup:", optimal_set , len(optimal_set))
    return optimal_set

In [ ]:
# G = nx.petersen_graph()
G = load_gcol('grafovi\gcol\gcol1.txt')
# G = nx.Graph()
# G.add_edges_from([(1,2),(1,3),(1,5),(1,6),(2,4),(2,6),(3,5),(3,4),(4,5),(4,6),(5,6)])

shortest_paths = dict(nx.all_pairs_shortest_path_length(G))


rez = genetic_algorithm(G,shortest_paths)
print(is_resolving_set(rez,G))

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_173372\1498853304.py:2: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol1.txt')


Generacija 1, Najbolji fitnes: 32
Generacija 2, Najbolji fitnes: 26
Generacija 3, Najbolji fitnes: 18
Generacija 4, Najbolji fitnes: 13
Generacija 5, Najbolji fitnes: 11
Generacija 6, Najbolji fitnes: 10
Generacija 7, Najbolji fitnes: 11
Generacija 8, Najbolji fitnes: 10
Generacija 9, Najbolji fitnes: 10
Generacija 10, Najbolji fitnes: 11
Generacija 11, Najbolji fitnes: 10
Generacija 12, Najbolji fitnes: 10
Generacija 13, Najbolji fitnes: 10
Generacija 14, Najbolji fitnes: 11
Generacija 15, Najbolji fitnes: 11

Optimalni rešavajući skup: {3, 44, 47, 79, 80, 18, 21, 23, 28, 63, 31} 11
True
